# Looking at the algorithm performance

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re

In [ ]:
# Path for output of plots
output_path = '../../data/output/results'

In [ ]:
# Path to the DataFrame file
dataframe_path = '../../data/output/plus_algorithm/results_algorithm_performance.pd'

In [ ]:
# Load the DataFrame
plus_algorithm_results = pd.read_pickle(dataframe_path)
print(plus_algorithm_results)

In [ ]:
num_entries = plus_algorithm_results.shape[0]
print(f"Number of entries in the DataFrame: {num_entries}")

## 1 - Update some columns: competition, athlete number, ground truth ...

In [ ]:
# Define file paths
input_dataframe_path = "../../data/output/plus_algorithm/results_algorithm_performance.pd"
output_dataframe_path = "../../data/output/plus_algorithm/updated_results_algorithm_performance.pd"

# Load the DataFrame
plus_algorithm_results = pd.read_pickle(input_dataframe_path)

# Function to extract details from 'video_name'
def extract_details(video_name):
    parts = video_name.split("_")
    
    # Determine competition
    competition = "lenzburg" if "lenzburg" in parts else "villars" if "villars" in parts else None
    
    # Determine gender
    gender = "male" if "men" in parts else "female" if "women" in parts else None

    # Extract athlete number (e.g., n11, n103, etc.)
    athlete_number_match = re.search(r'n(\d+)', video_name)
    athlete_number = athlete_number_match.group(1) if athlete_number_match else None

    # Extract ground truth (last part of the name, could be a number or number with +)
    ground_truth_match = re.search(r'(\d+\+?|\d+)', parts[-1])
    ground_truth = ground_truth_match.group(1) if ground_truth_match else None

    return pd.Series([competition, athlete_number, gender, ground_truth])

# Apply extraction function
plus_algorithm_results[['competition', 'athlete_number', 'gender', 'ground_truth']] = plus_algorithm_results['video_name'].apply(extract_details)

# Save the updated DataFrame
plus_algorithm_results.to_pickle(output_dataframe_path)

print("Updated DataFrame has been saved to:", output_dataframe_path)

In [ ]:
dataframe_path = '../../data/output/plus_algorithm/updated_results_algorithm_performance.pd'

# Load the DataFrame
updated_plus_algorithm_results = pd.read_pickle(dataframe_path)
print(updated_plus_algorithm_results)

In [ ]:
# now we add the ground truth plus column
# Define file path
dataframe_path = '../../data/output/plus_algorithm/updated_results_algorithm_performance.pd'

# Load the DataFrame
updated_plus_algorithm_results = pd.read_pickle(dataframe_path)

# Create 'ground_truth_plus' column (True if "+" in ground_truth, False otherwise)
updated_plus_algorithm_results["ground_truth_plus"] = updated_plus_algorithm_results["ground_truth"].astype(str).str.contains("\+")

# Save the updated DataFrame
updated_plus_algorithm_results.to_pickle(dataframe_path)

print("Updated DataFrame with 'ground_truth_plus' has been saved to:", dataframe_path)

# Display first few rows to verify
print(updated_plus_algorithm_results.head())

## 2 - Gaining insights about the results and data

In [ ]:
# Ensure final_score and ground_truth are treated as strings for proper comparison
updated_plus_algorithm_results["final_score"] = updated_plus_algorithm_results["final_score"].astype(str)
updated_plus_algorithm_results["ground_truth"] = updated_plus_algorithm_results["ground_truth"].astype(str)

# Count number of athletes by gender
gender_counts = updated_plus_algorithm_results["gender"].value_counts()

# Count number of videos by competition
competition_counts = updated_plus_algorithm_results["competition"].value_counts()

# Count number of scores with "+" and "noplus"
plus_counts = updated_plus_algorithm_results["ground_truth_plus"].value_counts()

# Count how many times final_score matches ground_truth
matching_scores = (updated_plus_algorithm_results["final_score"] == updated_plus_algorithm_results["ground_truth"]).sum()

# Count total number of rows
total_rows = len(updated_plus_algorithm_results)

# Count how many scores do NOT match ground_truth
non_matching_scores = total_rows - matching_scores

# Print Insights
print("Dataset Insights:")
print(f"Female Athletes: {gender_counts.get('female', 0)}")
print(f"Male Athletes: {gender_counts.get('male', 0)}")
print(f"Videos from Lenzburg: {competition_counts.get('lenzburg', 0)}")
print(f"Videos from Villars: {competition_counts.get('villars', 0)}")
print(f"Scores with '+': {plus_counts.get(True, 0)}")
print(f"Scores with 'noplus': {plus_counts.get(False, 0)}")
print(f"Final Score matches Ground Truth: {matching_scores}")
print(f"Final Score does NOT match Ground Truth: {non_matching_scores}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the DataFrame
dataframe_path = "../../data/output/plus_algorithm/updated_results_algorithm_performance.pd"
updated_plus_algorithm_results = pd.read_pickle(dataframe_path)

# Capitalize competition names
updated_plus_algorithm_results["competition"] = updated_plus_algorithm_results["competition"].str.capitalize()

# Count number of videos by competition and gender
competition_gender_counts = updated_plus_algorithm_results.groupby(["competition", "gender"]).size().unstack()

# Set Seaborn style for a clean scientific look
sns.set_style("white")

# Create bar plot
plt.figure(figsize=(8, 5))
# competition_gender_counts.plot(kind="bar", stacked=True, width=0.6, color=["black", "darkgray"])
competition_gender_counts.plot(kind="bar", stacked=True, width=0.6, color=["#4E79A7", "#76B7B2"])

# Add numbers inside the bars
for i, (female, male) in enumerate(zip(competition_gender_counts["female"].fillna(0), 
                                       competition_gender_counts["male"].fillna(0))):
    # Add female count in white (inside the dark bar) if greater than 0
    if female > 0:
        plt.text(i, female / 2, str(int(female)), ha='center', va='center', fontsize=12, color="white", weight="bold")

    # Add male count in black (inside the light bar) if greater than 0
    if male > 0:
        plt.text(i, female + (male / 2), str(int(male)), ha='center', va='center', fontsize=12, color="black", weight="bold")

# competition_gender_counts.plot(kind="bar", stacked=True, width=0.6, color=["black", "darkgray"], edgecolor="black")
# Customize plot appearance
plt.title("Number of Videos by Competition and Gender", fontsize=16)
plt.xlabel("Competition", fontsize=14)
plt.yticks(fontsize=12)
plt.ylabel("Number of Videos", fontsize=14)
plt.xticks(rotation=0, fontsize=12)  # Keep competition labels horizontal
plt.legend(labels=["Female", "Male"], title_fontsize=12, fontsize=12)

# Remove the grid
plt.grid(False)

plt.tight_layout()

plt.savefig(f"{output_path}/number_of_videos_by_competition_and_gender_v2.png", dpi=300, bbox_inches="tight")
print(f"Plot saved: number_of_videos_by_competition_and_gender_v2.png")

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the DataFrame
dataframe_path = "../../data/output/plus_algorithm/updated_results_algorithm_performance.pd"
updated_plus_algorithm_results = pd.read_pickle(dataframe_path)

# Ensure the "plus" column is treated as boolean (True/False)
updated_plus_algorithm_results["ground_truth_plus"] = updated_plus_algorithm_results["ground_truth_plus"].astype(bool)

# Count occurrences of True (Plus) and False (No Plus) and normalize to percentage
plus_counts = updated_plus_algorithm_results["ground_truth_plus"].value_counts(normalize=True) * 100  # Convert to percentage

# Assign correct labels based on actual order
labels = ["Plus", "No Plus"] if plus_counts.index[0] else ["No Plus", "Plus"]

# Define correct colors: No Plus = Orange, Plus = Teal
colors = ["teal", "orange"] if plus_counts.index[0] else ["orange", "teal"]

# Create a donut chart with percentage labels moved outward
plt.figure(figsize=(7, 7))
wedges, texts, autotexts = plt.pie(
    plus_counts, labels=labels, autopct='%1.1f%%',
    colors=colors, startangle=90, wedgeprops={'edgecolor': 'white'},
    textprops={'fontsize': 12, 'weight': 'bold', 'color': 'black'}, pctdistance=0.77
)

# Add a smaller white circle at the center to create the "donut" effect
centre_circle = plt.Circle((0, 0), 0.55, fc="white")
plt.gca().add_artist(centre_circle)

plt.title("Distribution of Scores with and without a Plus", fontsize=16, y=0.95)  # Move title closer

plt.tight_layout()

# Save plot
plt.savefig(f"{output_path}/plus_distribution_over_all_videos.png", dpi=300, bbox_inches="tight")
print(f"Plot saved: plus_distribution_over_all_videos.png")

# Show plot
plt.show()

# Print the actual counts to verify correctness
print(plus_counts)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the DataFrame
dataframe_path = "../../data/output/plus_algorithm/updated_results_algorithm_performance.pd"
updated_plus_algorithm_results = pd.read_pickle(dataframe_path)

# Ensure final_score and ground_truth are treated as strings for comparison
updated_plus_algorithm_results["final_score"] = updated_plus_algorithm_results["final_score"].astype(str)
updated_plus_algorithm_results["ground_truth"] = updated_plus_algorithm_results["ground_truth"].astype(str)

# Create a column for whether the final score matches ground truth
updated_plus_algorithm_results["match"] = updated_plus_algorithm_results["final_score"] == updated_plus_algorithm_results["ground_truth"]

# Count correct and incorrect identifications by competition and gender
match_counts = updated_plus_algorithm_results.groupby(["competition", "gender", "match"]).size().unstack(fill_value=0)

# Ensure consistent ordering of categories
match_counts = match_counts.reindex(columns=[True, False])  # True = Match, False = No Match

# Custom x-axis labels with line breaks
custom_labels = [
    "Lenzburg\nWomen's Final",
    "Villars\nWomen's Semifinal",
    "Villars\nMen's Semifinal"
]

# Create stacked bar plot
plt.figure(figsize=(8, 6))
ax = match_counts.plot(kind="bar", stacked=True, width=0.6, 
                       color=["#006D77", "#83C5BE"], ax=plt.gca())

# Adjust y-axis range
plt.ylim(0, 18)

# Set custom y-axis ticks
custom_yticks = [1, 3, 5, 7, 9, 11, 13, 15]  # No ticks above 15
# Customize y-axis labels (increase font size)
plt.yticks(custom_yticks, fontsize=12)

# Customize x-axis labels
plt.xticks(ticks=range(len(custom_labels)), labels=custom_labels, rotation=0, fontsize=12)

# Enable only horizontal grid lines with dashed style and transparency
plt.grid(visible=False)  # Disable all grids first
plt.grid(axis="y", linestyle="--", alpha=0.5)  # Enable only horizontal grid


# Add percentages inside the "Correctly Identified" section of the bars
for i, (correct, incorrect) in enumerate(zip(match_counts[True], match_counts[False])):
    total = correct + incorrect
    if total > 0:
        percentage = (correct / total) * 100
        plt.text(i, correct / 2, f"{int(percentage)}%", ha='center', va='center', fontsize=12, weight='bold', color="white")

# Customize plot appearance
plt.title("Score Identification by Competition and Gender", fontsize=16, y=1.02)
# plt.xlabel("Competition and Route", fontsize=14)
plt.xlabel("", fontsize=14)
plt.ylabel("Number of Videos for Scoring", fontsize=14)
plt.legend(labels=["Correct Score Output", "Incorrect Score Output"], loc="upper left", title_fontsize=12, fontsize=12)
# plt.legend(title="Score identified", labels=["Correct", "Incorrect"], loc="upper left", title_fontsize=12, fontsize=12)
plt.tight_layout()

# Save plot
plt.savefig(f"{output_path}/score_correctness_over_routes_v2.png", dpi=300, bbox_inches="tight")
print(f"Plot saved: score_correctness_over_routes_v2.png")

# Show the plot
plt.show()

## Looking at the distribution of scores at each competition and the videos that were preprocessed

In [ ]:
# Load the DataFrame
dataframe_path = '../../data/output/plus_algorithm/updated_results_algorithm_performance.pd'
results_analysis = pd.read_pickle(dataframe_path)

# Drop NaN values in ground_truth before conversion
results_analysis = results_analysis.dropna(subset=["ground_truth"])

# Convert ground_truth to integer values (remove '+') for plotting
results_analysis["ground_truth_int"] = results_analysis["ground_truth"].astype(str).str.replace("+", "", regex=False).astype(int)

# Convert athlete_number to numeric for correct sorting
results_analysis["athlete_number"] = pd.to_numeric(results_analysis["athlete_number"], errors='coerce')

# Identify which ground_truth values contain '+'
results_analysis["has_plus"] = results_analysis["ground_truth"].astype(str).str.contains("\+")

# Define competition and gender combinations for plotting with specific score ranges
plot_configs = [
    ("villars", "female", 1, 51, "Score Distribution in Villars Semifinals - Women"),
    ("villars", "male", 1, 44, "Score Distribution in Villars Semifinals - Men"),
    ("lenzburg", "female", 1, 46, "Score Distribution in Lenzburg Finals - Women"),
]

# Generate separate scatter plots for each competition and gender
for competition, gender, score_min, score_max, plot_title in plot_configs:
    subset = results_analysis[
        (results_analysis["competition"] == competition) & 
        (results_analysis["gender"] == gender)
    ].sort_values("athlete_number")  # Ensure athletes are sorted

    if not subset.empty:
        plt.figure(figsize=(12, 6))

        # Create categorical x-axis labels (equal spacing)
        subset["athlete_index"] = range(len(subset))  # Creates a sequential index for equal spacing

        # Define color mapping for legend
        color_map = {True: "teal", False: "orange"}
        point_colors = subset["has_plus"].map(color_map)

        # Create scatter plot with equal spacing for athlete numbers
        scatter = sns.scatterplot(x=subset["athlete_index"], y=subset["ground_truth_int"], 
                                  s=100, palette=color_map, hue=subset["has_plus"], legend=True)

        # Rename legend labels
        legend_labels = {True: "Plus", False: "No Plus"}
        handles, labels = scatter.get_legend_handles_labels()
        labels = [legend_labels[eval(label)] for label in labels]  # Rename labels
        plt.legend(handles, labels, title="Progression", loc="upper right", title_fontsize=12, fontsize=12)

        # Annotate each point below the point
        for i, row in subset.iterrows():
            plt.text(row["athlete_index"], row["ground_truth_int"] - 1.5, str(row["ground_truth"]),
                     horizontalalignment='center', verticalalignment='top', fontsize=12, color='black')

        # Set y-axis limits to match the given competition range
        plt.ylim(score_min, score_max)

        # Ensure y-axis uses steps of 5
        plt.yticks(np.arange(score_min, score_max + 1, 5))

        # Ensure all athlete numbers are equally spaced
        plt.xticks(subset["athlete_index"], labels=subset["athlete_number"].astype(str), fontsize=12)

        # Improve layout
        plt.title(plot_title, fontsize=16)
        plt.xlabel("Athlete Number", fontsize=14)
        plt.ylabel("Score", fontsize=14)
        plt.yticks(fontsize=12)
        plt.grid(True, linestyle="--", alpha=0.25)
        plt.tight_layout()

        # **Save the plot**
        filename = f"{output_path}/scoring_distribution_{competition}_{gender}.png"
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        plt.show()

        print(f"Plot saved: {filename}")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch

# Load the DataFrame
dataframe_path = '../../data/output/plus_algorithm/updated_results_algorithm_performance.pd'
results_analysis = pd.read_pickle(dataframe_path)

# Drop NaN values in ground_truth before conversion
results_analysis = results_analysis.dropna(subset=["ground_truth"])

# Convert ground_truth to integer values (remove '+') for plotting
results_analysis["ground_truth_int"] = results_analysis["ground_truth"].astype(str).str.replace("+", "", regex=False).astype(int)

# Convert athlete_number to numeric for correct sorting
results_analysis["athlete_number"] = pd.to_numeric(results_analysis["athlete_number"], errors='coerce')

# Identify which ground_truth values contain '+'
results_analysis["has_plus"] = results_analysis["ground_truth"].astype(str).str.contains("\+")

# Define competition and gender combinations for plotting with specific score ranges
plot_configs = [
    ("villars", "female", 1, 51, "Score Distribution in Villars Women's Semifinals"),
    ("villars", "male", 1, 44, "Score Distribution in Villars Men's Semifinals"),
    ("lenzburg", "female", 1, 46, "Score Distribution in Lenzburg Women's Finals"),
]

# Generate separate bar plots for each competition and gender
for competition, gender, score_min, score_max, plot_title in plot_configs:
    subset = results_analysis[
        (results_analysis["competition"] == competition) & 
        (results_analysis["gender"] == gender)
    ].sort_values("ground_truth_int", ascending=True)  # Sort by score, lowest to highest

    if not subset.empty:
        num_bars = len(subset)

        # **Ensure consistent spacing & height across all plots**
        bar_height = 0.7  # Fixed height for uniform bars
        num_bars = len(subset)
        fig_height = max(5, num_bars * 0.5)  # Scale dynamically but ensure a minimum height

        # Create figure with dynamic height
        plt.figure(figsize=(10, fig_height))

        # Define color mapping
        color_map = {True: "teal", False: "orange"}
        bar_colors = subset["has_plus"].map(color_map)

        # Create horizontal bar plot with fixed bar height
        bars = plt.barh(subset["athlete_number"].astype(str), subset["ground_truth_int"], 
                        color=bar_colors, height=bar_height)

        # Explicitly set Y-Ticks to maintain even spacing
        plt.gca().set_yticks(range(len(subset)))
        plt.gca().set_yticklabels(subset["athlete_number"].astype(str), fontsize=12)

        # Annotate each bar with the actual ground_truth (with '+')
        for bar, (score, plus) in zip(bars, zip(subset["ground_truth"], subset["has_plus"])):
            plt.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2, str(score),
                     verticalalignment='center', fontsize=12, color="black")

        # Set x-axis limits to match the given competition range
        plt.xlim(score_min, score_max + 2)  # Adding small padding for text visibility

        # Ensure x-axis uses steps of 5
        plt.xticks(np.arange(score_min, score_max + 1, 5), fontsize=12)
        plt.yticks(fontsize=12)

        # Customize grid and labels
        plt.title(plot_title, fontsize=16)
        plt.xlabel("Score", fontsize=14)
        plt.ylabel("Bib Number (Athlete)", fontsize=14)

        # Create legend manually
        legend_patches = [Patch(color="teal", label="Plus"), Patch(color="orange", label="No Plus")]
        plt.legend(handles=legend_patches, loc="lower right", fontsize=12)

        # Improve layout
        plt.tight_layout()

        # **Save the plot**
        filename = f"{output_path}/scoring_distribution_barplot_{competition}_{gender}.png"
        plt.savefig(filename, dpi=300, bbox_inches="tight")
        plt.show()

        print(f"Plot saved: {filename}")

## Looking at how the algorithm performance progressed

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define the data for algorithm progression
data = {
    "Version": [
        "First Version: Baseline",
        "Fall Detection Improvement",
        "Duohold Implementation"
    ],
    "Correctly Identified": [15, 20, 27],
    "Total": [30, 30, 30]  # Total attempts per version
}

# Create DataFrame
progression_df = pd.DataFrame(data)
progression_df["Accuracy (%)"] = (progression_df["Correctly Identified"] / progression_df["Total"]) * 100

# Define x-axis positions with better alignment
x_positions = np.arange(len(progression_df))

# Create line plot
plt.figure(figsize=(9, 6))
plt.plot(x_positions, progression_df["Accuracy (%)"], marker="o", linestyle="-", 
         color="teal", linewidth=2, markersize=8)

# Add percentage labels next to each point
for i, percentage in enumerate(progression_df["Accuracy (%)"]):
    plt.text(x_positions[i], percentage + 3, f"{int(percentage)}%",  # Increased spacing from +2 to +3
             ha='center', fontsize=12, weight='bold', color="black")

# Adjust x-axis limits to add spacing on both sides
plt.xlim(-0.5, len(progression_df) - 0.5)  # Adds 0.5 space on both sides

# Adjust x-axis labels alignment
plt.xticks(x_positions, progression_df["Version"], fontsize=12, ha='center')

# Set y-axis range from 0 to 100%
plt.ylim(0, 105)
plt.yticks(np.arange(0, 101, 20), fontsize=12)

# Customize grid (horizontal dashed lines only)
plt.grid(axis="y", linestyle="--", alpha=0.5)

# Customize plot appearance
plt.title("Algorithm Performance Over Versions", fontsize=16, y=1.02)
plt.xlabel("", fontsize=14)
plt.ylabel("Accuracy (%)", fontsize=14)

# Improve layout
plt.tight_layout()

# Save plot
plt.savefig(f"{output_path}/algorithm_progression_accuracy_lineplot.png", dpi=300, bbox_inches="tight")
print(f"Plot saved: algorithm_progression_accuracy_lineplot.png")

# Show the plot
plt.show()

## Looking at the Confusion Matrix for detecting a plus

Compute the confusion matrix (True Positives, False Positives, False Negatives, True Negatives).

Calculate performance metrics:
- Precision = TP / (TP + FP) → How many predicted pluses are actually correct?
- Recall (Sensitivity) = TP / (TP + FN) → How many actual pluses were identified?
- F1-score = 2 * (Precision * Recall) / (Precision + Recall) → Harmonic mean of precision and recall
- Accuracy = (TP + TN) / (Total Samples) → Overall correctness

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score

# Load the DataFrame
dataframe_path = "../../data/output/plus_algorithm/updated_results_algorithm_performance.pd"
updated_plus_algorithm_results = pd.read_pickle(dataframe_path)

# Convert 'plus' and 'ground_truth_plus' to boolean values (if not already)
updated_plus_algorithm_results["plus"] = updated_plus_algorithm_results["plus"].astype(bool)
updated_plus_algorithm_results["ground_truth_plus"] = updated_plus_algorithm_results["ground_truth_plus"].astype(bool)

# Compute Confusion Matrix
cm = confusion_matrix(updated_plus_algorithm_results["ground_truth_plus"], updated_plus_algorithm_results["plus"])

# Extract values
TN, FP, FN, TP = cm.ravel()

# Calculate Performance Metrics
precision = precision_score(updated_plus_algorithm_results["ground_truth_plus"], updated_plus_algorithm_results["plus"])
recall = recall_score(updated_plus_algorithm_results["ground_truth_plus"], updated_plus_algorithm_results["plus"])
f1 = f1_score(updated_plus_algorithm_results["ground_truth_plus"], updated_plus_algorithm_results["plus"])
accuracy = accuracy_score(updated_plus_algorithm_results["ground_truth_plus"], updated_plus_algorithm_results["plus"])

# Print Metrics
print(f"Confusion Matrix:\n{cm}")
print(f"Precision: {precision:.2f}")
print(f"Recall (Sensitivity): {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"Accuracy: {accuracy:.2f}")

# Plot Confusion Matrix with "crest" color palette
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap=sns.light_palette("teal", as_cmap=True), 
            xticklabels=["No Plus Pred", "Plus Pred"],
            yticklabels=["No Plus True", "Plus True"],
            annot_kws={"size": 16})  # Bigger numbers inside plot

# Customize labels
plt.xlabel("Predicted Label", fontsize=13, fontweight="bold")
plt.ylabel("True Label", fontsize=13, fontweight="bold")
plt.title("Confusion Matrix for Plus Detection", fontsize=15)

# Customize tick labels
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

# Show plot
plt.tight_layout()

# Save plot
plt.savefig(f"{output_path}/confusion_matrix_plus.png", dpi=300, bbox_inches="tight")
print(f"Plot saved: confusion_matrix_plus.png")

plt.show()

## Making some tabels to have an overview of the data and results

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score
from IPython.display import display

# Load the DataFrame
dataframe_path = "../../data/output/plus_algorithm/updated_results_algorithm_performance.pd"
updated_plus_algorithm_results = pd.read_pickle(dataframe_path)

# Ensure 'plus' and 'ground_truth_plus' are boolean
updated_plus_algorithm_results["plus"] = updated_plus_algorithm_results["plus"].astype(bool)
updated_plus_algorithm_results["ground_truth_plus"] = updated_plus_algorithm_results["ground_truth_plus"].astype(bool)

# Define routes
routes = [
    ("lenzburg", "female", "Women's Final Route"),
    ("villars", "female", "Women's Semifinal Route"),
    ("villars", "male", "Men's Semifinal Route"),
]

# **Manually provided Score Identification Accuracy values**
score_identification_accuracy = ["100%", "90%", "86%"]

# Store metrics for each route
table_data = []
for idx, (competition, gender, route_name) in enumerate(routes):
    subset = updated_plus_algorithm_results[
        (updated_plus_algorithm_results["competition"] == competition) &
        (updated_plus_algorithm_results["gender"] == gender)
    ]
    
    # Number of Videos
    num_videos = len(subset)

    # Plus Detection Metrics
    num_pluses = subset["ground_truth_plus"].sum()  # True pluses
    detected_pluses = subset["plus"].sum()  # Predicted pluses
    
    precision = precision_score(subset["ground_truth_plus"], subset["plus"], zero_division=0) * 100 if num_videos > 0 else 0
    recall = recall_score(subset["ground_truth_plus"], subset["plus"], zero_division=0) * 100 if num_videos > 0 else 0
    f1 = f1_score(subset["ground_truth_plus"], subset["plus"], zero_division=0) * 100 if num_videos > 0 else 0
    
    # Plus Accuracy (how many actual pluses were identified correctly)
    plus_accuracy = ((subset["plus"] & subset["ground_truth_plus"]).sum() / num_pluses * 100) if num_pluses > 0 else 0
    
    # Append data
    table_data.append([
        num_videos, 
        score_identification_accuracy[idx],  # Manually provided accuracy
        num_pluses, 
        detected_pluses, 
        f"{precision:.0f}%", 
        f"{recall:.0f}%", 
        f"{f1:.0f}%", 
        f"{plus_accuracy:.0f}%"  # Plus Accuracy
    ])

# Convert to DataFrame
columns = [
    "Number of Videos", 
    "Score Identification Accuracy", 
    "Number of Pluses", 
    "Detected Pluses", 
    "Precision", 
    "Recall", 
    "F1-score", 
    "Plus Accuracy"
]
table_df = pd.DataFrame(table_data, columns=columns)

# **Create MultiIndex DataFrame for column grouping (Lenzburg & Villars)**
columns = pd.MultiIndex.from_tuples([
    ("Lenzburg", "Women's Final Route"), 
    ("Villars", "Women's Semifinal Route"), 
    ("Villars", "Men's Semifinal Route")
])
table_df = table_df.T  # Transpose for structured display
table_df.columns = columns  # Assign MultiIndex columns

# **Row Grouping: "Overall" and "Plus" sections**
row_labels = [
    ("Overall", "Number of Videos"),
    ("Overall", "Score Identification Accuracy"),
    ("Plus", "Number of Pluses"),
    ("Plus", "Detected Pluses"),
    ("Plus", "Precision"),
    ("Plus", "Recall"),
    ("Plus", "F1-score"),
    ("Plus", "Plus Accuracy"),
]

# Assign new MultiIndex for rows
table_df.index = pd.MultiIndex.from_tuples(row_labels)

# **Style the table to center-align headers and values, no background colors**
styled_table = table_df.style.set_table_styles([
    {"selector": "th", "props": [("text-align", "center"), ("border-bottom", "1px solid black")]}  # **Add thin black line**
]).set_properties(**{"text-align": "center"})
# Style the table to center-align headers and values, with border lines
styled_table = table_df.style.set_table_styles([
    {"selector": "th", "props": [("text-align", "center"), ("border-bottom", "1px solid black")]},  # Header line
    {"selector": "td", "props": [("text-align", "center"), ("border-bottom", "1px solid black")]}  # Row line
]).set_properties(**{"text-align": "center"})

# Display the styled table
display(styled_table)

In [ ]:
import pandas as pd
from IPython.display import display

# Define MultiIndex columns
columns = pd.MultiIndex.from_tuples([
    ("Lenzburg", "Women's Finale Route"),
    ("Villars", "Women's Semifinal Route"),
    ("Villars", "Men's Semifinal Route")
])

# Define the table data
data = [
    ["5", "10", "15"],    # Number of Videos
    ["100%", "90%", "86%"] # Accuracy
]

# Create DataFrame
table_df = pd.DataFrame(data, columns=columns, index=["Number of Videos", "Accuracy"])

# Style the DataFrame to center-align the headers
styled_table = table_df.style.set_table_styles([
    {"selector": "th", "props": [("text-align", "center")]}
]).set_properties(**{"text-align": "center"})

# Display the styled table
display(styled_table)

In [ ]:
# Store metrics for each route (Plus-specific metrics)
table_data_plus = []
for idx, (competition, gender, route_name) in enumerate(routes):
    subset = updated_plus_algorithm_results[
        (updated_plus_algorithm_results["competition"] == competition) &
        (updated_plus_algorithm_results["gender"] == gender)
    ]
    
    # Plus Detection Metrics
    num_pluses = subset["ground_truth_plus"].sum()  # True pluses
    detected_pluses = subset["plus"].sum()  # Predicted pluses
    
    precision = precision_score(subset["ground_truth_plus"], subset["plus"], zero_division=0) * 100 if num_pluses > 0 else 0
    recall = recall_score(subset["ground_truth_plus"], subset["plus"], zero_division=0) * 100 if num_pluses > 0 else 0
    f1 = f1_score(subset["ground_truth_plus"], subset["plus"], zero_division=0) * 100 if num_pluses > 0 else 0
    
    # Plus Accuracy (how many actual pluses were identified correctly)
    plus_accuracy = ((subset["plus"] & subset["ground_truth_plus"]).sum() / num_pluses * 100) if num_pluses > 0 else 0
    
    # Append data
    table_data_plus.append([
        num_pluses, 
        detected_pluses, 
        f"{precision:.0f}%", 
        f"{recall:.0f}%", 
        f"{f1:.0f}%", 
        f"{plus_accuracy:.0f}%"  # Plus Accuracy
    ])

# Convert to DataFrame for the Plus table
columns_plus = [
    "Number of Pluses", 
    "Detected Pluses", 
    "Precision", 
    "Recall", 
    "F1-score", 
    "Plus Accuracy"
]
table_df_plus = pd.DataFrame(table_data_plus, columns=columns_plus)

# Define MultiIndex columns
columns = pd.MultiIndex.from_tuples([
    ("Lenzburg", "Women's Finale Route"),
    ("Villars", "Women's Semifinal Route"),
    ("Villars", "Men's Semifinal Route")
])

table_df_plus = table_df_plus.T  # Transpose for structured display
table_df_plus.columns = columns  # Assign MultiIndex columns

# **Style the table to center-align the headers and values**
styled_table_plus = table_df_plus.style.set_table_styles([
    {"selector": "th", "props": [("text-align", "center")]}  # Center align headers
]).set_properties(**{"text-align": "center"})

# Display the styled Plus table
display(styled_table_plus)